In [ ]:
import pandas as pd

# 파일 경로 리스트
file_paths = [
    "movie_details/kobis_boxoffice_with_details_2015_2016.csv",
    "movie_details/kobis_boxoffice_with_details_2017_2018.csv",
    "movie_details/kobis_boxoffice_with_details_2019_2020.csv",
    "movie_details/kobis_boxoffice_with_details_2021_2022.csv",
    "movie_details/kobis_boxoffice_with_details_2023_2025.csv"
]

# 데이터프레임 리스트로 읽어오기
dfs = [pd.read_csv(fp, encoding='utf-8') for fp in file_paths]

# 합치기 (row-wise)
merged_df = pd.concat(dfs, ignore_index=True)

# 중복 제거 (필요하다면 'movieCd' 기준 등 원하는 컬럼명으로)
merged_df = merged_df.drop_duplicates(subset=['movieCd'])

# 인덱스 초기화
merged_df.reset_index(drop=True, inplace=True)

# 저장
save_path = "movie_details/kobis_boxoffice_with_details_merged2.csv"
merged_df.to_csv(save_path, index=False, encoding='utf-8-sig')

print(f"합친 파일 저장 경로: {save_path}")


합친 파일 저장 경로: movie_details/kobis_boxoffice_with_details_merged2.csv


In [20]:

# 파일 읽기
df = pd.read_csv("movie_details/kobis_boxoffice_with_details_merged.csv", encoding='utf-8')

# showRange에서 시작 날짜 추출 후 'YYYYMMDD'로 변환
import numpy as np

# 파일 읽기 (이미 df가 있다고 가정)
# df = pd.read_csv(...)

df['targetDt'] = np.where(
    df['showRange'].notnull(),
    df['showRange'].str.split('~').str[0].str.replace('-', ''),
    df['targetDt']
)
df['targetDt'] = df['targetDt'].str.replace('-', '')


# 저장(원한다면)
df.to_csv("movie_details/kobis_boxoffice_with_details_with_targetDt.csv", index=False, encoding='utf-8-sig')

C:\Users\kevib\AppData\Local\Temp\ipykernel_31168\1430491816.py:2: DtypeWarning: Columns (0,17,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("movie_details/kobis_boxoffice_with_details_merged.csv", encoding='utf-8')


In [ ]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('kobis_boxoffice_with_details_merged.csv')

# openDt_box를 datetime으로 변환
df['openDt_box'] = pd.to_datetime(df['openDt_box'], format='%Y-%m-%d', errors='coerce')

# 2015년 1월 1일 이후 데이터만 필터링
df = df[df['openDt_box'] >= '2015-01-01']

# 지정한 열만 선택
keep_columns = ['movieCd', 'movieNm_box', 'actors', 'directors', 'genres', 'nations', 
                'repNationNm', 'productionCompanies', 'salesAcc', 'audiAcc','showTm']
df = df[keep_columns]

# 중복 제거 및 salesAcc, audiAcc 최대값 유지
grouped_df = df.groupby('movieCd').agg({
    'salesAcc': 'max',
    'audiAcc': 'max',
    **{col: 'first' for col in df.columns if col not in ['movieCd', 'salesAcc', 'audiAcc']}
}).reset_index()

# 결과 저장
grouped_df.to_csv('kobis_boxoffice_with_details_final.csv', index=False)

# 결과 확인
print(grouped_df.head())

    movieCd     salesAcc  audiAcc movieNm_box  \
0  20098169  40601955671  4620936      쥬라기 월드   
1  20100211   2029362300   273264      다이노 타임   
2  20130021   3317041989   425752   협녀, 칼의 기억   
3  20130566   3194054331   406139         무뢰한   
4  20131262   7353194469   948101         허삼관   

                                       actors  directors          genres  \
0  크리스 프랫|주디 그리어|브라이스 달라스 하워드|케이티 맥그라스|제이크 존슨   콜린 트레보로우      액션|어드벤처|SF   
1       멜라니 그리피스|제인 린치|윌리엄 볼드윈|스티븐 볼드윈|롭 슈나이더  존 카프카|최윤석  애니메이션|어드벤처|코미디   
2                         이병헌|전도연|김고은|이경영|김태우        박흥식           사극|액션   
3                         김남길|전도연|박성웅|곽도원|강태영        오승욱          멜로/로맨스   
4                          하정우|하지원|전혜진|장광|주진모        하정우             드라마   

  nations  repNationNm productionCompanies  showTm  
0      미국          NaN                None   124.0  
1   한국|미국          NaN            주식회사 토이온    85.0  
2      한국          NaN          (주)티피에스컴퍼니   120.0  
3      한국          NaN           (주)

C:\Users\srdyh\AppData\Local\Temp\ipykernel_14148\2846866848.py:4: DtypeWarning: Columns (0,18,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('kobis_boxoffice_with_details_merged.csv')
